In [ ]:
import numpy as np

# Energy parameters in Joules
E_max = 162000  # Battery capacity in Joules
E_t = 50 / 1000  # Transmission energy in Joules
E_s = 10 / 1000  # Sensing energy in Joules
E_w = 10 / 1000  # Wake-up energy in Joules
E_0 = 1 / 1000   # Sleep energy in Joules

def run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M=5, L=5000):
    polling_counts = {mote: 0 for mote in range(1, num_nodes + 1)}  # Track polling counts for each node
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    time_since_last_poll = {i: 0 for i in range(1, num_nodes + 1)}
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}

    for t in range(len(pivot_df)):
        # Update the state of each node using dEWMA
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)

        # Update the time since each node was last polled
        for mote in range(1, num_nodes + 1):
            time_since_last_poll[mote] = t - last_update_times[mote]

        whittle_indices = {}
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            delta_t_dynamic = t - last_update_times[mote]
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            future_aoii_passive = predict_node_aoii_from_sink(state_sink[f'mote{mote}'], delta_t_dynamic + 1)
            future_aoii_active = 0 if np.random.rand() < transmission_probs[mote - 1] else future_aoii_passive
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + penalty
            whittle_indices[mote] = q_passive - q_active

        nodes_to_poll = [mote for mote in whittle_indices if whittle_indices[mote] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        for mote in range(1, num_nodes + 1):
            if (t - last_update_times[mote]) > L and mote not in nodes_to_poll:
                if len(nodes_to_poll) > 0:
                    lowest_whittle_node = min(nodes_to_poll, key=lambda x: whittle_indices[x])
                    nodes_to_poll.remove(lowest_whittle_node)
                nodes_to_poll.append(mote)

        # Poll the selected nodes and update their state
        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            delta_t_dynamic = t - last_update_times[mote]  # Time since last update

            # Update AoII and DEWMA state for polled node
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            aoii_history[f'mote{mote}'].append(current_aoii)  # Record AoII
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)
            state_sink[column_name] = state_node[column_name]

            # Update polling count and last polling time
            polling_counts[mote] += 1
            last_update_times[mote] = t

    # Calculate polling fraction (fw) and average lifetime
    time_steps = len(pivot_df)
    fw = {mote: polling_counts[mote] / time_steps for mote in polling_counts}

    average_lifetime_hours = np.mean([
        E_max / (fw[mote] * (E_t + E_s + E_w) + (1 - fw[mote]) * E_0) for mote in fw
    ]) / 3600
    average_lifetime_years = average_lifetime_hours / 8760  # Convert hours to years

    print(f"Average Sensor Lifetime (years) for Whittle AoII: {average_lifetime_years:.2f}")
    return average_lifetime_years

# Example usage
transmission_probs = np.full(num_nodes, 1.0)  # Assume perfect transmission success
M = 5  # Number of nodes to poll at each step
average_lifetime_whittle_aoii = run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M)
